In [1]:
import requests
import json
import pandas as pd
import itertools
import time
import math
import numpy as np
from tqdm import tqdm

In [18]:
api_key = 'RGAPI-f8d576b9-66ee-4d2c-892d-11b1beb7c782'

In [15]:
IRON_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/IV?page='
IRON_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/III?page='
IRON_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/II?page='
IRON_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/I?page='
BRONZE_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/IV?page='
BRONZE_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/III?page='
BRONZE_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/II?page='
BRONZE_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/I?page='
SILVER_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/IV?page='
SILVER_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/III?page='
SILVER_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/II?page='
SILVER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page='
GOLD_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page='
GOLD_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/III?page='
GOLD_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/II?page='
GOLD_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/I?page='
PLATINUM_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/IV?page='
PLATINUM_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/III?page='
PLATINUM_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/II?page='
PLATINUM_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/I?page='
DIAMOND_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/IV?page='
DIAMOND_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/III?page='
DIAMOND_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/II?page='
DIAMOND_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page='
MASTER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/MASTER/I?page='
GRANDMASTER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page='
CHALLENGER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page='
MATCH_URL = 'https://kr.api.riotgames.com/lol/match/v4/matches/'

In [16]:
def summonersT(tier_url, api_key):
    print('----------------------------------------------------Summoners start----------------------------------------------------')
    summoners = []
    for i in range(1,5):
        res = requests.get(tier_url+str(i), headers={"X-Riot-Token": api_key})
        summoners.append(json.loads(res.text)[5:105])
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    print('Waiting for summoners end...')
    time.sleep(120)
    print('----------------------------------------------------Summoners done-----------------------------------------------------')
    return pd.DataFrame(list(itertools.chain(*summoners)))['summonerId'].values

def getAccountId(summoners, api_key):
    print('\n----------------------------------------------------Accounts start-----------------------------------------------------')
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    accountS = []
    time.sleep(2)
    a = 0
    for i  in tqdm(summoners):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetSummonerId = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/'+str(i)
        res = requests.get(togetSummonerId, headers={"X-Riot-Token": api_key})
        accountS.append(json.loads(res.text))
    print('\n-----------------------------------------------------Accounts done-----------------------------------------------------')   
    return pd.DataFrame(accountS)['accountId'].values

def getMatch(encryptedAccountId, api_key):
    print('\n-----------------------------------------------------MatchId start-----------------------------------------------------')
    a = 0
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    objects = []
    matchIds = []
    time.sleep(2)
    for i in tqdm(encryptedAccountId):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetMatchOb = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?endIndex=1&beginIndex=0'.format(str(i))
        res = requests.get(togetMatchOb, headers={"X-Riot-Token": api_key})
        ob = pd.DataFrame(json.loads(res.text))
        if 'matches' in list(ob):
            objects.append(pd.DataFrame(ob)['matches'].values[0])
        else:
            print(ob)
            print("----Error recovery time----")
            time.sleep(90)
            rest=getMatchRe([i], api_key)
            matchIds.append(rest)
            print("----Error fixed----")
    matchIds = matchIds + list(pd.DataFrame(objects)['gameId'].values)
    print('\n-----------------------------------------------------MatchId done-----------------------------------------------------')   
    return matchIds


def getMatchRe(encryptedAccountId, api_key):
    a = 0
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    objects = []
    matchIds = []
    time.sleep(2)
    for i in tqdm(encryptedAccountId):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetMatchOb = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?endIndex=1&beginIndex=0'.format(str(i))
        res = requests.get(togetMatchOb, headers={"X-Riot-Token": api_key})
        ob = pd.DataFrame(json.loads(res.text))
        if 'matches' in list(ob):
            objects.append(pd.DataFrame(ob)['matches'].values[0])
        else:
            print(ob)
            print("----Error recovery time----")
            time.sleep(90)
            rest=getMatchRe([i], api_key)
            matchIds.append(rest)
    matchIds = matchIds + list(pd.DataFrame(objects)['gameId'].values)
    return matchIds[0]


def getMatchId(tier, api_key):
    summoners = summonersT(tier, api_key)
    accountId = getAccountId(summoners, api_key)
    gameId = getMatch(accountId, api_key)
    return gameId

In [5]:
iron_4_gameId = getMatchId(IRON_URL4, api_key)
iron_3_gameId = getMatchId(IRON_URL3, api_key)
iron_2_gameId = getMatchId(IRON_URL2, api_key)
iron_1_gameId = getMatchId(IRON_URL1, api_key)

----------------------------------------------------Summoners start----------------------------------------------------
min: 59 sec: 10
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 1 sec: 10


100%|██████████| 400/400 [10:00<00:00,  1.50s/it]  



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 11 sec: 12


 35%|███▌      | 140/400 [02:58<02:49,  1.54it/s]  

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 15 sec: 42


...

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 28 sec: 58



 70%|██████▉   | 278/400 [07:30<57:06, 28.09s/it]

----Error fixed----


100%|██████████| 400/400 [12:26<00:00,  1.87s/it]  


----------------------------------------------------MatchId done----------------------------------------------------


In [6]:
np.save('iron_1_gameId', iron_1_gameId)
np.save('iron_2_gameId', iron_2_gameId)
np.save('iron_3_gameId', iron_3_gameId)
np.save('iron_4_gameId', iron_4_gameId)

In [ ]:
bronze_4_gameId = getMatchId(BRONZE_URL4, api_key)
bronze_3_gameId = getMatchId(BRONZE_URL3, api_key)
bronze_2_gameId = getMatchId(BRONZE_URL2, api_key)
bronze_1_gameId = getMatchId(BRONZE_URL1, api_key)

In [57]:
np.save('bronze_4_gameId', bronze_4_gameId)
np.save('bronze_3_gameId', bronze_3_gameId)
np.save('bronze_2_gameId', bronze_2_gameId)
np.save('bronze_1_gameId', bronze_1_gameId)

In [ ]:
silver_4_gameId = getMatchId(SILVER_URL4, api_key)
silver_3_gameId = getMatchId(SILVER_URL3, api_key)
silver_2_gameId = getMatchId(SILVER_URL2, api_key)
silver_1_gameId = getMatchId(SILVER_URL1, api_key)

In [ ]:
np.save('silver_4_gameId', silver_4_gameId)
np.save('silver_3_gameId', silver_3_gameId)
np.save('silver_2_gameId', silver_2_gameId)
np.save('silver_1_gameId', silver_1_gameId)

In [6]:
gold_4_gameId = getMatchId(GOLD_URL4, api_key)
gold_3_gameId = getMatchId(GOLD_URL3, api_key)
gold_2_gameId = getMatchId(GOLD_URL2, api_key)
gold_1_gameId = getMatchId(GOLD_URL1, api_key)

----------------------------------------------------Summoners start----------------------------------------------------
min: 49 sec: 44
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 51 sec: 44


100%|██████████| 400/400 [10:02<00:00,  1.51s/it]  



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 1 sec: 48


 29%|██▉       | 116/400 [02:49<02:17,  2.07it/s]  

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 6 sec: 11


...


----------------------------------------------------MatchId done----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 0 sec: 29
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 2 sec: 29


100%|██████████| 400/400 [10:01<00:00,  1.50s/it]  



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 12 sec: 33


100%|██████████| 400/400 [10:52<00:00,  1.63s/it]  


----------------------------------------------------MatchId done----------------------------------------------------


In [7]:
np.save('gold_4_gameId', gold_4_gameId)
np.save('gold_3_gameId', gold_3_gameId)
np.save('gold_2_gameId', gold_2_gameId)
np.save('gold_1_gameId', gold_1_gameId)

In [9]:
platinum_4_gameId = getMatchId(PLATINUM_URL4, api_key)
platinum_3_gameId = getMatchId(PLATINUM_URL3, api_key)
platinum_2_gameId = getMatchId(PLATINUM_URL2, api_key)
platinum_1_gameId = getMatchId(PLATINUM_URL1, api_key)

----------------------------------------------------Summoners start----------------------------------------------------
min: 47 sec: 17
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 49 sec: 17


100%|██████████| 400/400 [10:04<00:00,  1.51s/it]  



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 59 sec: 24


100%|██████████| 400/400 [10:55<00:00,  1.64s/it]  



----------------------------------------------------MatchId done----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 10 sec: 23
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 12 sec: 23


...


----------------------------------------------------MatchId done----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 57 sec: 55
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 59 sec: 53


100%|██████████| 400/400 [09:58<00:00,  1.50s/it]  



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 9 sec: 54


100%|██████████| 400/400 [10:56<00:00,  1.64s/it]  


----------------------------------------------------MatchId done----------------------------------------------------


In [10]:
np.save('platinum_4_gameId', platinum_4_gameId)
np.save('platinum_3_gameId', platinum_3_gameId)
np.save('platinum_2_gameId', platinum_2_gameId)
np.save('platinum_1_gameId', platinum_1_gameId)

In [11]:
diamond_4_gameId = getMatchId(DIAMOND_URL4, api_key)
diamond_3_gameId = getMatchId(DIAMOND_URL3, api_key)
diamond_2_gameId = getMatchId(DIAMOND_URL2, api_key)
diamond_1_gameId = getMatchId(DIAMOND_URL1, api_key)
master_gameId = getMatchId(MASTER_URL1, api_key)
grandmaster_gameId = getMatchId(GRANDMASTER_URL1, api_key)
challenger_gameId = getMatchId(CHALLENGER_URL1, api_key)

----------------------------------------------------Summoners start----------------------------------------------------
min: 36 sec: 21
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 38 sec: 21


100%|██████████| 400/400 [09:59<00:00,  1.50s/it]  



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 48 sec: 22


100%|██████████| 400/400 [10:50<00:00,  1.63s/it]  



----------------------------------------------------MatchId done----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 59 sec: 16
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 1 sec: 16


...


----------------------------------------------------MatchId done----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 31 sec: 17
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 33 sec: 17


100%|██████████| 13/13 [00:02<00:00,  4.42it/s]



-----------------------------------------------------Accounts done-----------------------------------------------------

----------------------------------------------------MatchId start----------------------------------------------------
min: 33 sec: 22


100%|██████████| 13/13 [00:04<00:00,  2.84it/s]


----------------------------------------------------MatchId done----------------------------------------------------


In [12]:
np.save('diamond_4_gameId', diamond_4_gameId)
np.save('diamond_3_gameId', diamond_3_gameId)
np.save('diamond_2_gameId', diamond_2_gameId)
np.save('diamond_1_gameId', diamond_1_gameId)
np.save('master_gameId', master_gameId)
np.save('grandmaster_gameId', grandmaster_gameId)
np.save('challenger_gameId', challenger_gameId)

In [12]:
a = np.load('diamond\diamond_1_gameId.npy')
len(set(a.tolist()))

353

In [79]:
def chamFind(i):
    if i == 266:
        print("Aatrox, id : {}".format(i))
    elif i == 412:  
        print("Thresh, id : {}".format(i))
    elif i == 23:
        print( "Tryndamere, id : {}".format(i))
    elif i == 79:
        print( "Gragas, id : {}".format(i))
    elif i == 69:
        print( "Cassiopeia, id : {}".format(i))
    elif i == 136:
        print( "Aurelion Sol, id : {}".format(i))
    elif i == 13:
        print( "Ryze, id : {}".format(i))
    elif i == 78:
        print( "Poppy, id : {}".format(i))
    elif i == 14:
        print( "Sion, id : {}".format(i))
    elif i == 1:
        print( "Annie, id : {}".format(i))
    elif i == 202:
        print( "Jhin, id : {}".format(i))
    elif i == 43:
        print( "Karma, id : {}".format(i))
    elif i == 111:
        print( "Nautilus, id : {}".format(i))
    elif i == 240:
        print( "Kled, id : {}".format(i))
    elif i == 99:
        print( "Lux, id : {}".format(i))
    elif i == 103:
        print( "Ahri, id : {}".format(i))
    elif i == 2:
        print( "Olaf, id : {}".format(i))
    elif i == 112:
        print( "Viktor, id : {}".format(i))
    elif i == 34:
        print( "Anivia, id : {}".format(i))
    elif i == 27:
        print( "Singed, id : {}".format(i))
    elif i == 86:
        print( "Garen, id : {}".format(i))
    elif i == 127:
        print( "Lissandra, id : {}".format(i))
    elif i == 57:
        print( "Maokai, id : {}".format(i))
    elif i == 25:
        print( "Morgana, id : {}".format(i))
    elif i == 28:
        print( "Evelynn, id : {}".format(i))
    elif i == 105:
        print( "Fizz, id : {}".format(i))
    elif i == 74:
        print( "Heimerdinger, id : {}".format(i))
    elif i == 238:
        print( "Zed, id : {}".format(i))
    elif i == 68:
        print( "Rumble, id : {}".format(i))
    elif i == 82:
        print( "Mordekaiser, id : {}".format(i))
    elif i == 37:
        print( "Sona, id : {}".format(i))
    elif i == 96:
        print( "Kog'Maw, id : {}".format(i))
    elif i == 55:
        print( "Katarina, id : {}".format(i))
    elif i == 117:
        print( "Lulu, id : {}".format(i))
    elif i == 22:
        print( "Ashe, id : {}".format(i))
    elif i == 30:
        print( "Karthus, id : {}".format(i))
    elif i == 12:
        print( "Alistar, id : {}".format(i))
    elif i == 122:
        print( "Darius, id : {}".format(i))
    elif i == 67:
        print( "Vayne, id : {}".format(i))
    elif i == 110:
        print( "Varus, id : {}".format(i))
    elif i == 77:
        print( "Udyr, id : {}".format(i))
    elif i == 89:
        print( "Leona, id : {}".format(i))
    elif i == 126:
        print( "Jayce, id : {}".format(i))
    elif i == 134:
        print( "Syndra, id : {}".format(i))
    elif i == 80:
        print( "Pantheon, id : {}".format(i))
    elif i == 92:
        print( "Riven, id : {}".format(i))
    elif i == 121:
        print( "Kha'Zix, id : {}".format(i))
    elif i == 42:
        print( "Corki, id : {}".format(i))
    elif i == 268:
        print( "Azir, id : {}".format(i))
    elif i == 51:
        print( "Caitlyn, id : {}".format(i))
    elif i == 76:
        print( "Nidalee, id : {}".format(i))
    elif i == 85:
        print( "Kennen, id : {}".format(i))
    elif i == 3:
        print( "Galio, id : {}".format(i))
    elif i == 45:
        print( "Veigar, id : {}".format(i))
    elif i == 432:
        print( "Bard, id : {}".format(i))
    elif i == 150:
        print( "Gnar, id : {}".format(i))
    elif i == 90:
        print( "Malzahar, id : {}".format(i))
    elif i == 104:
        print( "Graves, id : {}".format(i))
    elif i == 254:
        print( "Vi, id : {}".format(i))
    elif i == 10:
        print( "Kayle, id : {}".format(i))
    elif i == 39:
        print( "Irelia, id : {}".format(i))
    elif i == 64:
        print( "Lee Sin, id : {}".format(i))
    elif i == 420:
        print( "Illaoi, id : {}".format(i))
    elif i == 60:
        print( "Elise, id : {}".format(i))
    elif i == 106:
        print( "Volibear, id : {}".format(i))
    elif i == 20:
        print( "Nunu, id : {}".format(i))
    elif i == 4:
        print( "Twisted Fate, id : {}".format(i))
    elif i == 24:
        print( "Jax, id : {}".format(i))
    elif i == 102:
        print( "Shyvana, id : {}".format(i))
    elif i == 429:
        print( "Kalista, id : {}".format(i))
    elif i == 36:
        print( "Dr. Mundo, id : {}".format(i))
    elif i == 427:
        print( "Ivern, id : {}".format(i))
    elif i == 131:
        print( "Diana, id : {}".format(i))
    elif i == 223:
        print( "Tahm Kench, id : {}".format(i))
    elif i == 63:
        print( "Brand, id : {}".format(i))
    elif i == 113:
        print( "Sejuani, id : {}".format(i))
    elif i == 8:
        print( "Vladimir, id : {}".format(i))
    elif i == 154:
        print( "Zac, id : {}".format(i))
    elif i == 421:
        print( "Rek'Sai, id : {}".format(i))
    elif i == 133:
        print( "Quinn, id : {}".format(i))
    elif i == 84:
        print( "Akali, id : {}".format(i))
    elif i == 163:
        print( "Taliyah, id : {}".format(i))
    elif i == 18:
        print( "Tristana, id : {}".format(i))
    elif i == 120:
        print( "Hecarim, id : {}".format(i))
    elif i == 15:
        print( "Sivir, id : {}".format(i))
    elif i == 236:
        print( "Lucian, id : {}".format(i))
    elif i == 107:
        print( "Rengar, id : {}".format(i))
    elif i == 19:
        print( "Warwick, id : {}".format(i))
    elif i == 72:
        print( "Skarner, id : {}".format(i))
    elif i == 54:
        print( "Malphite, id : {}".format(i))
    elif i == 157:
        print( "Yasuo, id : {}".format(i))
    elif i == 101:
        print( "Xerath, id : {}".format(i))
    elif i == 17:
        print( "Teemo, id : {}".format(i))
    elif i == 75:
        print( "Nasus, id : {}".format(i))
    elif i == 58:
        print( "Renekton, id : {}".format(i))
    elif i == 119:
        print( "Draven, id : {}".format(i))
    elif i == 35:
        print( "Shaco, id : {}".format(i))
    elif i == 50:
        print( "Swain, id : {}".format(i))
    elif i == 91:
        print( "Talon, id : {}".format(i))
    elif i == 40:
        print( "Janna, id : {}".format(i))
    elif i == 115:
        print( "Ziggs, id : {}".format(i))
    elif i == 245:
        print( "Ekko, id : {}".format(i))
    elif i == 61:
        print( "Orianna, id : {}".format(i))
    elif i == 114:
        print( "Fiora, id : {}".format(i))
    elif i == 9:
        print( "Fiddlesticks, id : {}".format(i))
    elif i == 31:
        print( "Cho'Gath, id : {}".format(i))
    elif i == 33:
        print( "Rammus, id : {}".format(i))
    elif i == 7:
        print( "LeBlanc, id : {}".format(i))
    elif i == 16:
        print( "Soraka, id : {}".format(i))
    elif i == 26:
        print( "Zilean, id : {}".format(i))
    elif i == 56:
        print( "Nocturne, id : {}".format(i))
    elif i == 222:
        print( "Jinx, id : {}".format(i))
    elif i == 83:
        print( "Yorick, id : {}".format(i))
    elif i == 6:
        print( "Urgot, id : {}".format(i))
    elif i == 203:
        print( "Kindred, id : {}".format(i))
    elif i == 21:
        print( "Miss Fortune, id : {}".format(i))
    elif i == 62:
        print( "Wukong, id : {}".format(i))
    elif i == 53:
        print( "Blitzcrank, id : {}".format(i))
    elif i == 98:
        print( "Shen, id : {}".format(i))
    elif i == 201:
        print( "Braum, id : {}".format(i))
    elif i == 5:
        print( "Xin Zhao, id : {}".format(i))
    elif i == 29:
        print( "Twitch, id : {}".format(i))
    elif i == 11:
        print( "Master Yi, id : {}".format(i))
    elif i == 44:
        print( "Taric, id : {}".format(i))
    elif i == 32:
        print( "Amumu, id : {}".format(i))
    elif i == 41:
        print( "Gangplank, id : {}".format(i))
    elif i == 48:
        print( "Trundle, id : {}".format(i))
    elif i == 38:
        print( "Kassadin, id : {}".format(i))
    elif i == 161:
        print( "Vel'Koz, id : {}".format(i))
    elif i == 143:
        print( "Zyra, id : {}".format(i))
    elif i == 267:
        print( "Nami, id : {}".format(i))
    elif i == 59:
        print( "Jarvan IV, id : {}".format(i))
    elif i == 81:
        print( "Ezreal, id : {}".format(i))



In [13]:
import tqdm

In [80]:
for i in range(1, 1000):
    chamFind(i)

Annie, id : 1
Olaf, id : 2
Galio, id : 3
Twisted Fate, id : 4
Xin Zhao, id : 5
Urgot, id : 6
LeBlanc, id : 7
Vladimir, id : 8
Fiddlesticks, id : 9
Kayle, id : 10
Master Yi, id : 11
Alistar, id : 12
Ryze, id : 13
Sion, id : 14
Sivir, id : 15
Soraka, id : 16
Teemo, id : 17
Tristana, id : 18
Warwick, id : 19
Nunu, id : 20
Miss Fortune, id : 21
Ashe, id : 22
Tryndamere, id : 23
Jax, id : 24
Morgana, id : 25
Zilean, id : 26
Singed, id : 27
Evelynn, id : 28
Twitch, id : 29
Karthus, id : 30
Cho'Gath, id : 31
Amumu, id : 32
Rammus, id : 33
Anivia, id : 34
Shaco, id : 35
Dr. Mundo, id : 36
Sona, id : 37
Kassadin, id : 38
Irelia, id : 39
Janna, id : 40
Gangplank, id : 41
Corki, id : 42
Karma, id : 43
Taric, id : 44
Veigar, id : 45
Trundle, id : 48
Swain, id : 50
Caitlyn, id : 51
Blitzcrank, id : 53
Malphite, id : 54
Katarina, id : 55
Nocturne, id : 56
Maokai, id : 57
Renekton, id : 58
Jarvan IV, id : 59
Elise, id : 60
Orianna, id : 61
Wukong, id : 62
Brand, id : 63
Lee Sin, id : 64
Vayne, id : 6

In [65]:
a = np.load('diamond\diamond_1_gameId.npy')
print(a)

[4938079235 4938032240 4934432603 4937970678 4937969699 4938438709
 4937775143 4938474987 4937716042 4935541210 4938071143 4932725430
 4936940766 4928329119 4938073106 4935793743 4938381551 4938376141
 4929506050 4937924048 4935190430 4937836182 4938375776 4937601956
 4936346560 4934545174 4935205919 4938459727 4938423692 4930364611
 4938490626 4937727279 4927237766 4937415070 4938425510 4938501676
 4916577436 4936488519 4937784927 4930874321 4927638952 4937804815
 4938458138 4930551606 4937865326 4938057520 4936874407 4935641308
 4938019068 4938042200 4938057520 4936623976 4937554538 4938375078
 4938436066 4934650902 4937844096 4937728911 4937813463 4938478239
 4936809672 4938458603 4936880322 4938390797 4938454028 4938388166
 4930822383 4938067069 4937858798 4938032240 4930793385 4935209147
 4934496997 4930874321 4938478332 4934813344 4938475427 4934525538
 4937832595 4938370555 4937477499 4936274853 4935830415 4938389188
 4938396928 4937962627 4936592393 4937805677 4938524178 493213

In [ ]:
matches=[]
for i in a:
    load_url = MATCH_URL + str(i)
    res = requests.get(load_url, headers={"X-Riot-Token": api_key})
    matches.append(json.loads(res.text))
print(matches)

In [ ]:
temp = [[],[],[],[],[],[],[],[],[],[]]
for i in matches:

    #     for k, v in i.items():
#         print(k, v)
#         print()
    for j in i['participants']:
        for k,v in j['stats'].items():
            temp.append(v)
#         for jk, jv in i['stats'].items():
#             print(jk, jv)
        
    print()
    print()
